In [5]:
import pandas as pd
import numpy as np
import random

In [2]:
%store -r weather_data # 0-45 is precip, 46-91 is max temp, etc

In [23]:
clay = random.uniform(0, 100)
sand = random.uniform(0, 100 - clay)
silt = 100 - clay - sand
soil_parameters = np.array([clay, sand, silt])
soil_parameters

array([94.28715792,  3.12686928,  2.5859728 ])

In [ ]:
yields = np.array([231.9,228.4,222.1,219.7,219.1,217.3,216.7,206.1,202.8,201.2,200.0,197.3,213.6])
nitrogen = np.array([40])